## Relevant Links
- Examples: https://docs.google.com/spreadsheets/d/14pdiODt9PmsD2F___Lpv2rO-RByTtC8PYcEfxL8iIuY/edit#gid=186467173
- LangChain: https://github.com/hwchase17/langchain
- Chroma: no docker or API_KEY - https://www.trychroma.com/
- OpenAI account usage: https://platform.openai.com/account/usage

## Tasks

- **Eric & Jacob**: Validation (`validate_stages.py` is not yet working, and has not been incorporated)
- **Jacob**: Label class recognition
    - determine what classes the user is referring to for each label field
- **Allen**: Evaluation Identification Stage:
    - `links/evaluation_run_selector.py` 
    - add stage to `dataset_view_generator.py`
    - add `prompts/evaluation_task_rules.txt` prompt
    - add examples to Examples spreadsheet in new tab, and then put in `examples` folder
- **Leila**: Add support for `hardness` brain runs
    - fill out the template in `links/brain_run_selector.py` 
    - add `prompts/hardness_task_rules.txt` prompt modeled after uniqueness and mistakenness.
    - new examples tab `hardness` modeled after `uniqueness` and `mistakenness` tabs and then put in `examples` folder
- **Vini**: More examples
    - scrape examples from community Slack covering as wide a range of scenarios as possible
    - complex filters and view expressions!
    - videos
    - every type of label
    - varied naming conventions

## Additional notes

- Want to validate embedded fields
- At present, there is no support for multi-line Python code to generate views. It all needs to be done inline
- No support for groups, 3D/point clouds, `concat()`, or `mongo()`.

## Getting started

`pip install openai langchain chroma`

Then create an OpenAI account and generate an API key

`export OPENAI_API_KEY=...`

In [1]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

## Get similar prompts

In [2]:
from links.view_stage_example_selector import generate_view_stage_examples_prompt

Using embedded DuckDB without persistence: data will be transient


In [4]:
query = "Five random images from the dataset"

In [5]:
view_stage_examples_prompt = generate_view_stage_examples_prompt(dataset, query)

Getting or creating embeddings for queries...
Loading embeddings from file...
Saving embeddings to file...


In [6]:
print(view_stage_examples_prompt)

Generate code to produce the FiftyOne view stages for the following prompts:


Input: 10 random images with tables
Output: [match(
    F("ground_truth.detections.label").contains("table")
), take(10)]

Input: Take two random samples from the dataset
Output: [take(2)]

Input: 10 most unique images
Output: [sort_by("uniqueness"), limit(10)]

Input: show me the 20 most similar imagages to the first image
Output: [sort_by_similarity(dataset.first().id, k=20)]

Input: sort by similarity to image 10
Output: [sort_by_similarity(dataset.skip(9).id)]

Input: Images that only contain dogs
Output: [match(
    F("ground_truth.detections.label").is_subset(["dog"])
)]

Input: take 10 random samples from the dataset with seed=51
Output: [take(10, seed=51)]

Input: show me 50 random samples
Output: [take(50)]

Input: show me random samples
Output: [shuffle()]

Input: exclude the 8th image
Output: [exclude([dataset.skip(7).first().id)]

Input: Five random images from the dataset
Output:


## Generate View Stage Descriptions Prompt

In [7]:
from links.view_stage_description_selector import generate_view_stage_descriptions_prompt

In [8]:
view_stage_descriptions_prompt = generate_view_stage_descriptions_prompt(view_stage_examples_prompt)

In [9]:
print(view_stage_descriptions_prompt)

Here is some information about view stages you might want to use:


    View stage: take
    Description: Randomly samples the given number of samples from the collection
    Inputs: size: int (non-positive values result in an empty view), seed: optional random seed to use for sample selection

    

    View stage: match
    Description: Filters the samples in the collection by the given filter
    Inputs: filter: ViewExpression or MongoDB expression that returns a boolean describing the filter to apply

    

    View stage: skip
    Description: Omits the given number of samples from the head of the collection. Non-positive skip values result in no samples being omitted
    Inputs: skip: int

    

    View stage: sort_by_similarity
    Description: Sorts the collection by similarity to a specified query
    Inputs: query: ID or iterable of IDs or num_dims vector or num_queries x num_dims array of vectors or text prompt or iterable of text prompts, k: int (defaults to entire collect

## Brain method selector

In [10]:
from links.brain_method_selector import select_brain_methods

In [11]:
query = "Five most unique images from the dataset"
select_brain_methods(query)

['uniqueness']

In [12]:
query = "50 least unique images from the dataset that were hard"
select_brain_methods(query)

['uniqueness', 'hardness']

In [13]:
query = "Five most mistaken images from the dataset"
select_brain_methods(query)

['mistakenness']

## Brain run selector

In [14]:
from links.brain_run_selector import select_brain_runs

In [15]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [16]:
query = "most unique images from the dataset with key 'uniqueness1 that are similar to image 10"
select_brain_runs(dataset, query, ["uniqueness", "image_similarity"])

{'uniqueness': 'uniqueness1', 'image_similarity': 'img_sim'}

In [17]:
query = "Five most mistaken images from the dataset with ground truth field 'predictions"
select_brain_runs(dataset, query, ["mistakenness"])

{'mistakenness': 'mistakenness1'}

## Field selector

In [18]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [19]:
from links.field_selector import select_fields

In [20]:
query = "sort pred1 by number of detections"
fields = select_fields(dataset, query)
print(fields)

[predictions]


## Label class selector

In [21]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

dataset = foz.load_zoo_dataset("quickstart")

Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [22]:
from links.label_class_selector import select_label_classes

In [24]:
query = "sort Bear and glove predictions by number of detections"
prompt = select_label_classes(dataset, query, "[predictions]")
print(prompt)

Matching Bear with bear
Class name glove not found for label predictions
{'predictions': ['bear', 'glove']}


## DatasetView generator

In [29]:
from gpt_view_generator import get_gpt_view_text

In [30]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [31]:
query = "images with tables"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: images with tables
Identified likely view stages: ['match', 'sort_by_similarity', 'skip', 'exclude', 'exists', 'filter_labels', 'limit', 'select_fields', 'sort_by', 'take']
Identified brain methods: ['text_similarity']
Identified brain runs: {'text_similarity': 'clip'}
Identified potentially relevant fields: [ground_truth]
Class name table not found for label ground_truth
Identified label classes: {'ground_truth': ['table']}
[match(
    F("ground_truth.detections.label").contains("table")
)]


In [32]:
query = "high confidence detections"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: high confidence detections
Identified likely view stages: ['filter_labels', 'exists', 'match', 'filter_field', 'limit', 'match_labels', 'sort_by', 'exclude', 'exclude_by', 'exclude_fields']
Identified potentially relevant fields: [predictions]
Identified label classes: {'predictions': []}
[filter_labels("ground_truth", F("confidence") > 0.95)]


In [33]:
query = "first 30 images with a ground truth detection"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: first 30 images with a ground truth detection
Identified likely view stages: ['match', 'sort_by_similarity', 'exists', 'filter_field', 'filter_labels', 'limit_labels', 'select_fields', 'skip', 'sort_by', 'take']
Identified potentially relevant fields: [ground_truth]
Identified label classes: {'ground_truth': []}
[match(F("ground_truth.detections").length() > 0), take(30)]


In [34]:
query = "sort by image uniqueness1 for images with a >0.5 confidence classification"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: sort by image uniqueness1 for images with a >0.5 confidence classification
Identified likely view stages: ['exists', 'match', 'sort_by', 'sort_by_similarity', 'filter_labels', 'filter_keypoints', 'match_labels', 'skip', 'exclude', 'exclude_by']
Identified brain methods: ['uniqueness']
Identified brain runs: {'uniqueness': 'uniqueness1'}
Identified potentially relevant fields: [predictions, mistakenness1_eval_fp]
Identified label classes: {'predictions': []}
[match_labels(
    filter=F("predictions.confidence") > 0.5,
    fields="ground_truth"
), sort_by("uniqueness1")]


In [35]:
query = "find the images most resembling a farm scene"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Finding similar examples for query: find the images most resembling a farm scene
Identified likely view stages: ['match', 'sort_by_similarity', 'skip', 'exclude', 'filter_labels', 'limit', 'select_fields', 'sort_by', 'take', 'exclude_by']
Identified brain methods: ['image_similarity', 'text_similarity']
Identified brain runs: {'image_similarity': 'clip', 'text_similarity': 'img_sim'}
Identified potentially relevant fields: [ground_truth, predictions]
Identified label classes: {'ground_truth': [], 'predictions': []}
[sort_by_similarity("farm scene", brain_key="img_sim")]


## Roadmap

### Validation and error handling
- even fully fleshed out, this is not going to be 100% accurate. Need to validate that it is actually creating a valid DataSetView
- if it isn't, prompt the user for more specific information
    - this could potentially be tailored to which part of the process it failed

### Plugin
- Everything will be wrapped in a single python function `generate_datasetview_with_chatgpt(dataset, prompt)`
- Then need to turn this into a plugin. It will take the `session.dataset` as dataset, and will set `session.view`
- This will probably be a menu-item plugin - we could use the ChatGPT symbol - it will take user input
- Would love to have a toggle the user can specify for whether they want 
    - the view created from scratch, or 
    - the view stages concatenated with their existing view

### Future
- Memory/chat history
- More general question-answering:
    - First stage: determine whether the user is asking a question about the entire dataset, or individual samples
    - Use ChatGPT as a dispatcher, deciding what other models/processes to invoke.
        - if it is about aggregations, then decide what FiftyOne aggregation to perform, and interpret the results
        - if it is about a single image, employ BLIPv2 or equivalent...